In [89]:
import tensorflow as tf 
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [90]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [91]:
gpu = tf.config.list_physical_devices('GPU')
if gpu:
  # only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpu[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpu), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    print(e)

1 Physical GPUs, 1 Logical GPU


In [92]:
data = pd.read_csv("data3.csv")
data

,xAcc,yAcc,zAcc,xGyro,yGyro,zGyro,label
0,6.99,-0.57,-7.28,-2.75,-3.23,2.62,fall
1,6.51,-0.75,-6.93,-0.67,-6.35,4.64,fall
2,6.22,-0.63,-6.73,0.79,-5.49,3.85,fall
3,6.34,-0.62,-6.80,1.59,-2.26,0.67,fall
4,6.49,-0.39,-6.60,0.67,-0.24,-1.10,fall
...,...,...,...,...,...,...,...
96795,7.30,-1.16,-4.58,18.19,3.60,-35.77,light
96796,7.39,-0.37,-5.05,20.08,0.06,-34.55,light
96797,7.52,-1.46,-5.82,22.58,0.12,-28.02,light
96798,7.58,-2.14,-6.31,19.47,2.44,-22.52,light


In [93]:
new_data = data.drop("label",axis=1).to_numpy().reshape(-1, 400, 6)
labels = np.array(data.label.iloc[np.arange(0,data.shape[0], 400)])

In [97]:
new_data.shape

(242, 400, 6)

In [127]:
X_train, X_test, y_train, y_test = train_test_split(new_data, labels)
input_shape = X_train.shape[1:]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, seq_len*n_features))
X_test = scaler.fit_transform(X_test.reshape(-1, seq_len*n_features))

le = LabelEncoder()
le.fit(np.unique(y_train))
y_train = le.transform(y_train)
y_test = le.transform(y_test)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=7)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=7)

X_train = X_train.reshape(-1, seq_len, n_features) 
X_test = X_test.reshape(-1, seq_len, n_features)

In [112]:
X_train.shape

(181, 400, 6)

# First CNN

In [113]:
first_model = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='sigmoid')  
])

In [114]:
first_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [115]:
first_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 398, 32)           608       
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 199, 32)           0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 6368)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 64)                407616    
_________________________________________________________________
dense_19 (Dense)             (None, 7)                 455       
Total params: 408,679
Trainable params: 408,679
Non-trainable params: 0
_________________________________________________________________


In [117]:
first_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
6/6 [==============================] - 0s 28ms/step - loss: 1.7398e-04 - accuracy: 1.0000 - val_loss: 0.8026 - val_accuracy: 0.8197
Epoch 2/50
6/6 [==============================] - 0s 13ms/step - loss: 1.7082e-04 - accuracy: 1.0000 - val_loss: 0.8032 - val_accuracy: 0.8197
Epoch 3/50
6/6 [==============================] - 0s 11ms/step - loss: 1.6791e-04 - accuracy: 1.0000 - val_loss: 0.8041 - val_accuracy: 0.8197
Epoch 4/50
6/6 [==============================] - 0s 10ms/step - loss: 1.6522e-04 - accuracy: 1.0000 - val_loss: 0.8047 - val_accuracy: 0.8197
Epoch 5/50
6/6 [==============================] - 0s 9ms/step - loss: 1.6244e-04 - accuracy: 1.0000 - val_loss: 0.8056 - val_accuracy: 0.8197
Epoch 6/50
6/6 [==============================] - 0s 11ms/step - loss: 1.5965e-04 - accuracy: 1.0000 - val_loss: 0.8057 - val_accuracy: 0.8197
Epoch 7/50
6/6 [==============================] - 0s 11ms/step - loss: 1.5715e-04 - accuracy: 1.0000 - val_loss: 0.8065 - val_accuracy: 0.8197


In [84]:
first_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 17ms/step - loss: 1.8248 - accuracy: 0.7213


[1.8247588872909546, 0.7213114500045776]

## Deep Learning Nico

In [128]:
## (CNN-HE)

cnn_he = models.Sequential([
    layers.Conv1D(32, kernel_size=5, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(7, activation='softmax')
])

cnn_he.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

cnn_he.fit(X_train, y_train, epochs=70, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/70
3/3 [==============================] - 1s 123ms/step - loss: 3.4259 - accuracy: 0.2707 - val_loss: 3.3313 - val_accuracy: 0.3115
Epoch 2/70
3/3 [==============================] - 0s 44ms/step - loss: 1.8458 - accuracy: 0.4751 - val_loss: 1.2429 - val_accuracy: 0.6066
Epoch 3/70
3/3 [==============================] - 0s 30ms/step - loss: 0.8881 - accuracy: 0.6740 - val_loss: 1.0410 - val_accuracy: 0.5738
Epoch 4/70
3/3 [==============================] - 0s 31ms/step - loss: 0.7169 - accuracy: 0.7127 - val_loss: 0.7555 - val_accuracy: 0.7869
Epoch 5/70
3/3 [==============================] - 0s 28ms/step - loss: 0.4796 - accuracy: 0.8895 - val_loss: 0.6355 - val_accuracy: 0.7869
Epoch 6/70
3/3 [==============================] - 0s 30ms/step - loss: 0.3461 - accuracy: 0.8840 - val_loss: 0.5796 - val_accuracy: 0.7869
Epoch 7/70
3/3 [==============================] - 0s 32ms/step - loss: 0.2052 - accuracy: 0.9337 - val_loss: 0.5034 - val_accuracy: 0.7869
Epoch 8/70
3/3 [==========

In [129]:
## (CNN-3B3Conv) 

CNN_3B3Conv = models.Sequential([
    layers.Conv1D(128, kernel_size=4, activation='relu', input_shape=input_shape),
    layers.Conv1D(128, kernel_size=4, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='softmax')
    ])

CNN_3B3Conv.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
CNN_3B3Conv.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/20
3/3 [==============================] - 2s 261ms/step - loss: 1.8459 - accuracy: 0.2155 - val_loss: 1.5033 - val_accuracy: 0.4426
Epoch 2/20
3/3 [==============================] - 0s 20ms/step - loss: 1.3649 - accuracy: 0.5028 - val_loss: 1.0546 - val_accuracy: 0.7377
Epoch 3/20
3/3 [==============================] - 0s 21ms/step - loss: 0.8730 - accuracy: 0.7348 - val_loss: 0.8883 - val_accuracy: 0.8033
Epoch 4/20
3/3 [==============================] - 0s 21ms/step - loss: 0.7970 - accuracy: 0.7956 - val_loss: 0.7420 - val_accuracy: 0.7869
Epoch 5/20
3/3 [==============================] - 0s 23ms/step - loss: 0.6709 - accuracy: 0.7790 - val_loss: 0.7472 - val_accuracy: 0.7377
Epoch 6/20
3/3 [==============================] - 0s 22ms/step - loss: 0.4875 - accuracy: 0.8453 - val_loss: 0.6708 - val_accuracy: 0.8033
Epoch 7/20
3/3 [==============================] - 0s 23ms/step - loss: 0.4484 - accuracy: 0.8398 - val_loss: 0.6153 - val_accuracy: 0.8033
Epoch 8/20
3/3 [==========

In [133]:
## CNN-EDU
CNN_EDU = models.Sequential([
    layers.Conv1D(16, kernel_size=5, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(32, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='softmax')
])

CNN_EDU.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
CNN_EDU.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/100
3/3 [==============================] - 1s 165ms/step - loss: 1.8575 - accuracy: 0.1989 - val_loss: 1.6613 - val_accuracy: 0.2623
Epoch 2/100
3/3 [==============================] - 0s 48ms/step - loss: 1.6733 - accuracy: 0.2928 - val_loss: 1.5741 - val_accuracy: 0.5738
Epoch 3/100
3/3 [==============================] - 0s 36ms/step - loss: 1.5094 - accuracy: 0.4420 - val_loss: 1.4294 - val_accuracy: 0.5574
Epoch 4/100
3/3 [==============================] - 0s 32ms/step - loss: 1.3577 - accuracy: 0.5304 - val_loss: 1.1891 - val_accuracy: 0.5902
Epoch 5/100
3/3 [==============================] - 0s 24ms/step - loss: 1.1779 - accuracy: 0.6243 - val_loss: 1.0525 - val_accuracy: 0.6230
Epoch 6/100
3/3 [==============================] - 0s 26ms/step - loss: 0.9510 - accuracy: 0.7072 - val_loss: 0.9192 - val_accuracy: 0.7049
Epoch 7/100
3/3 [==============================] - 0s 25ms/step - loss: 0.8682 - accuracy: 0.7569 - val_loss: 0.7713 - val_accuracy: 0.7049
Epoch 8/100
3/3 [==